In [1]:
import numpy as np
import numba
import random
import math
from numba import jit, set_num_threads

def csr_spmv_single_thread(rowptr, colidx, val, x):
    """CSR format based SpMV (y=Ax)"""
    
    num_row = len(rowptr) - 1
    y = np.zeros(num_row, dtype=np.float32)
    
    for i in range(num_row):
        for j in range(rowptr[i], rowptr[i + 1]):
            Ax_data = val[j] * x[colidx[j]]
            y[i] += val[j] * x[colidx[j]]
    
    return y

@jit(nopython=True, parallel=True, nogil=True, fastmath=True)
def csr_spmv_multi_thread(y, num_row, rowptr, colidx, val, x):
    """CSR format based SpMV (y=Ax)"""
    
    for i in numba.prange(num_row):
        row_data = 0.0
        for j in range(rowptr[i], rowptr[i + 1]):
            Ax_data = val[j] * x[colidx[j]]
            row_data += val[j] * x[colidx[j]]
        y[i] = row_data
        
    return y

def sliced_ellpack_spmv_single_thread(N, slice_ptr, colidx, val, x, slice_height):
    """Sliced ELLPACK format based SpMV (y=Ax)"""
    
    y = np.zeros(N, dtype=np.float32)
    slice_count = int(N / slice_height)
    
    for s in range(slice_count):
        row_idx = s * slice_height
        for r in range(row_idx, row_idx + slice_height):
            for i in range(slice_ptr[s] + r - row_idx, slice_ptr[s + 1], slice_height):
                Ax_data = x[colidx[i]] * val[i]
                y[r] += Ax_data
    
    return y

@jit(nopython=True, parallel=True, nogil=True, fastmath=True)
def sliced_ellpack_spmv_multi_thread(y, N, slice_count, slice_ptr, colidx, val, x, slice_height):
    """Sliced ELLPACK format based SpMV (y=Ax)"""
    
    for s in numba.prange(slice_count):
        row_idx = s * slice_height
        loop_y = np.zeros(slice_height, dtype=np.float32)
        for r in range(slice_height):
            row_data = 0.0
            for i in range(slice_ptr[s] + r, slice_ptr[s + 1], slice_height):
                Ax_data = x[colidx[i]] * val[i]
                row_data += Ax_data
            loop_y[r] = row_data
        y[s * slice_height:(s + 1) * slice_height] = loop_y
        
    return y

def random_spmatrix(n_row, n_col, per_nnz):
    """Output a random value sparse matrix"""
    
    sp_matrix = []
    nnz_count = 0
    row_max_nnz = 0
    
    for i in range(n_row):
        row_data = []
        row_nnz_count = 0
        for j in range(n_col):
            r_val = random.randint(0, 100)
            if r_val < per_nnz:
                row_data.append(0)
            else:
                nnz_count += 1
                row_nnz_count += 1
                row_data.append(r_val)
        row_max_nnz = max(row_max_nnz, row_nnz_count)
        sp_matrix.append(row_data)
    
    return sp_matrix, nnz_count, row_max_nnz

def spmatrix_to_CSR(sp_matrix):
    """Convert sparse matrix to CSR format"""
    
    n_row = len(sp_matrix)
    n_col = len(sp_matrix[0])
    
    rowptr = []
    colidx = []
    val = []
    nnz_count = 0
    
    for i in range(n_row):
        rowptr.append(nnz_count)
        for j in range(n_col):
            if sp_matrix[i][j] != 0:
                nnz_count += 1
                colidx.append(j)
                val.append(sp_matrix[i][j])
    rowptr.append(nnz_count)
    
    return np.array(rowptr), np.array(colidx), np.array(val, dtype=np.float32)

def CSR_to_SELLPACK(rowptr, colidx, val, slice_height):
    """Convert CSR format to Sliced ELLPACK format"""
    
    N = len(rowptr) - 1 # number of rows
    slice_number = math.ceil(N / slice_height) # how many slices
    nnz_count = 0
    
    ell_colidx = []
    ell_sliceptr = []
    ell_val = []
    
    for i in range(slice_number):
        max_nnz = 0
        for s in range(slice_height):
            col_count = rowptr[i * slice_height + s + 1] - rowptr[i * slice_height + s]
            max_nnz = max(max_nnz, col_count)
        
        ell_sliceptr.append(nnz_count)
        for j in range(max_nnz):
            for k in range(slice_height):
                idx = i * slice_height + k # row index
                now_ptr = rowptr[idx] # start index of this row
                next_ptr = rowptr[idx + 1] # start index of next row
                nnz_count += 1 # count non-zero number
                if now_ptr + j < next_ptr:
                    ell_colidx.append(colidx[now_ptr + j])
                    ell_val.append(val[now_ptr + j])
                else:
                    ell_colidx.append(-1) # -1 means invalid
                    ell_val.append(0) # padded zero
    ell_sliceptr.append(nnz_count)
    
    return np.array(ell_colidx), np.array(ell_sliceptr), np.array(ell_val, dtype=np.float32)

In [2]:
# set number of rows, columns
n_row = 2000
n_col = 2000
# sparse matrix non-zero percentage
per_nnz = 5
# generate a sparse matrix fill with random value
sp_matrix, nnz_count, row_max_nnz = random_spmatrix(n_row, n_col, per_nnz)
nnz_per = (nnz_count / (n_row * n_col)) * 100
avg_nnz = nnz_count / n_row
print(str(nnz_count) + " non-zero elements in this sparse matrix (" + str(nnz_per) + "%).")
print("Row average non-zero elements: " + str(avg_nnz) + ", row max non-zero elements: " + str(row_max_nnz))

3802461 non-zero elements in this sparse matrix (95.061525%).
Row average non-zero elements: 1901.2305, row max non-zero elements: 1939


In [3]:
# convert sparse matrix to CSR format
csr_rowptr, csr_colidx, csr_val = spmatrix_to_CSR(sp_matrix)

In [14]:
# set slice height
slice_height = 2
# convert CSR to Sliced ELLPACK
ell_colidx, ell_sliceptr, ell_val = CSR_to_SELLPACK(csr_rowptr, csr_colidx, csr_val, slice_height)

In [5]:
# generate x array
x = np.ones(n_col, dtype=np.float64)
x *= 1.23
# generate data
csr_y = np.zeros(n_row, dtype=np.float32)
ell_y = np.zeros(n_row, dtype=np.float32)
slice_count = int(n_row / slice_height)

In [6]:
set_num_threads(4)

In [11]:
%timeit csr_output = csr_spmv_multi_thread(csr_y, n_row, csr_rowptr, csr_colidx, csr_val, x)

1.23 ms ± 36.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit ellpack_output = sliced_ellpack_spmv_multi_thread(ell_y, n_row, slice_count, ell_sliceptr, ell_colidx, ell_val, x, slice_height)

562 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
sum(csr_output - ellpack_output)

0.0

In [16]:
sliced_ellpack_spmv_multi_thread.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function sliced_ellpack_spmv_multi_thread, 
<ipython-input-1-4cb6c8eb78c8> (48)  


Parallel loop listing for  Function sliced_ellpack_spmv_multi_thread, <ipython-input-1-4cb6c8eb78c8> (48) 
-----------------------------------------------------------------------------------------------------|loop #ID
@jit(nopython=True, parallel=True, nogil=True, fastmath=True)                                        | 
def sliced_ellpack_spmv_multi_thread(y, N, slice_count, slice_ptr, colidx, val, x, slice_height):    | 
    """Sliced ELLPACK format based SpMV (y=Ax)"""                                                    | 
                                                                                                     | 
    for s in numba.prange(slice_count):--------------------------------------------------------------| #3
        row_idx = s * slice_height                                                                   | 
        loop_y = np.zeros(slice_hei

In [17]:
sliced_ellpack_spmv_multi_thread.inspect_types(pretty=True)

C:\Users\50621\anaconda3\lib\site-packages\numba\core\annotations\pretty_annotate.py:7: FutureWarning: The pretty_annotate functionality is experimental and might change API
  warn("The pretty_annotate functionality is experimental and might change API",


"48: @jit(nopython=True, parallel=True, nogil=True, fastmath=True)"
"49: def sliced_ellpack_spmv_multi_thread(y, N, slice_count, slice_ptr, colidx, val, x, slice_height):"
"50: """"""Sliced ELLPACK format based SpMV (y=Ax)"""""""
51:
label 0
"y = arg(0, name=y) :: array(float32, 1d, C)"
"N = arg(1, name=N) :: int64"
del N
"slice_count = arg(2, name=slice_count) :: int64"
"slice_ptr = arg(3, name=slice_ptr) :: array(int32, 1d, C)"
"colidx = arg(4, name=colidx) :: array(int32, 1d, C)"
